In [ ]:
!pip install boto3 --upgrade
!pip install sagemaker --upgrade

In [1]:
import boto3
import json
import sagemaker
from IPython.display import HTML

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
#Check Python version is greater than 3.8 which is required by Langchain
import sys
sys.version

'3.10.13 | packaged by conda-forge | (main, Oct 26 2023, 18:07:37) [GCC 12.3.0]'

In [3]:
sagemaker_session = sagemaker.Session()
sm_client = boto3.Session().client('sagemaker')

## Install and Setup

Execute the following commands from SageMaker Studio System terminal (Launcher --> System Terminal)

Setup Streamlit environemnt. Below assumes you have extracted code to /home/sagemaker-user/bedrock_onboarding directory

```bash
cd /home/sagemaker-user/bedrock_onboarding/applications
conda env create -f environment.yml
source activate bedrock-gen-ai-app
```

If you are running this in new SageMaker Stuido, perform following actions

```bash

Create a folder .streamlit under applications folder
mkdir .streamlit

In this folder create a file called config.toml and copy the following contents to that folder
touch config.toml

[server]
enableXsrfProtection = false
enableCORS = false

[browser]
gatherUsageStats = false
```
### Setup Environment variables
Setup the environment variables in .env file under application folder. You can setup Region, Authentication mechanism and other variables here.

## Authentication Setup with Cognito
You can optionally turn on a setting to require authentication for the application. It can authenticate against a Cognito Userpool. This application requires AppIntegration settings with Cognito user pool and you can configure to create AppIntegration and get the client secret ID. If you don't have a Cognito user pool you can create manually or using the given CloudFormation template. Follow instructions in [Cognito setup](CognitoSetup.md) to complete the setup. 

## Run application 
Run the following command to launch demo application. This runs the application to run on port 8081, restrict maximum file size for upload to 10MB. Both settings can be changed.

```bash
streamlit run Amazon_Bedrock.py --server.port 8081 --server.maxUploadSize 10
```

In [4]:
#Read Studio meta data to get Domain, User Profile details
with open('/opt/ml/metadata/resource-metadata.json','r') as cf:
    meta_data = json.load(cf)

studio_domain = meta_data['DomainId']
application_type = meta_data['AppType']
studio_region = sagemaker_session.boto_region_name
role_arn = sagemaker_session.get_caller_identity_arn()
studio_type = 'Studio' if application_type == 'JupyterLab' else 'Studio Classic'
domian_settings = sm_client.describe_domain(DomainId=studio_domain)
print(f'Studio Domain: {studio_domain}; Region: {studio_region}, Studio Type: {studio_type}')

Studio Domain: d-0mqhdu304zxs; Region: us-west-2, Studio Type: Studio


## Launch the application UI from SageMaker Studio Classic

If you are running this app from SageMaker studio, you need a proxy to run the application. Below code will help you lanuch the application after you have started it. Check the value of variable ``studio_type``

In [ ]:
user_profile_name = meta_data['UserProfileName']

In [ ]:
proxy_port = 8081
app_url = f'https://{studio_domain}.studio.{studio_region}.sagemaker.aws/jupyter/default/proxy/{proxy_port}/'
HTML(f"<a href='{app_url}'>Click to launch on SageMaker Studio Classic</a>")

## Launch the application UI from SageMaker Studio

If you are running this app from SageMaker studio, you need a proxy to run the application. 
Below code will help you lanuch the application after you have started it. Check the value of variable ``studio_type``

In [5]:
space_name = meta_data['SpaceName']
space_settings = sm_client.describe_space(
    DomainId=studio_domain,    
    SpaceName=space_name
)
user_profile_name = space_settings['OwnershipSettings']['OwnerUserProfileName']
studio_url = space_settings['Url']

In [ ]:
#If you are running this in a new SageMaker Studio, run the following
proxy_port = 8081
app_url = f'{studio_url}/proxy/{proxy_port}/'
HTML(f"<a href='{app_url}'>Click to launch on New SageMaker Studio</a>")

## Launch application UI locally
if you are running the application locally, you can navigate to following URL. 

In [ ]:
proxy_port = 8081
local_url = f'http://localhost:{proxy_port}/'
HTML(f"<a href='{local_url}'>Click to launch locally</a>")

## Deploy application to ECS
You can optionally deploy the Streamlit application to Amazon ECS service. 
Follow instructions in [Deploy to ECS](Deploy_to_ECS.md) to complete the setup. 

## Stop & Clean-up
To stop the application, go to terminal and kill the application Ctrl +C
Deactivate conda environment created to exit out

```bash
conda deactivate 
```

You can view all conda environments by running this command
```bash
conda env list
```

Remove the environment created

```bash
conda remove --name bedrock-gen-ai-app --all
```

To verify that the environment was removed, run
```bash
conda info --envs
```